Выбрать тест и проверить, есть  ли различия между выборками:

1 )  Даны две  независимые выборки. Не соблюдается условие нормальности
x1  380,420, 290
y1 140,360,200,900
Сделайте вывод по результатам, полученным с помощью функции

In [6]:
def mann_whitney_u_test(x, y):
    
    all_data = sorted(x + y)
    ranks = [sorted(all_data).index(value) + 1 + (all_data.count(value) - 1) / 2 for value in x + y]

    u_statistic_x = sum([rank for rank, sample in zip(ranks, x)])
    u_statistic_y = sum([rank for rank, sample in zip(ranks, y)])

    n_x = len(x)
    n_y = len(y)
    u_min = min(u_statistic_x, u_statistic_y)
    u_max = max(u_statistic_x, u_statistic_y)
    u_critical = min(u_min, n_x * n_y - u_max)


    critical_value = 5  

    print("Статистика U:", u_critical)
    print("Критическое значение U для уровня значимости 0.05:", critical_value)

x1 = [380, 420, 290]
y1 = [140, 360, 200, 900]

# тест Манна-Уитни
mann_whitney_u_test(x1, y1)


Статистика U: -3.0
Критическое значение U для уровня значимости 0.05: 5


2 ) Исследовалось влияние препарата на уровень давления пациентов. Сначала измерялось давление до приема препарата, потом через 10 минут и через 30 минут. Есть ли статистически значимые различия?

1е измерение до приема препарата: 150, 160, 165, 145, 155
2е измерение через 10 минут: 140, 155, 150,  130, 135
3е измерение через 30 минут: 130, 130, 120, 130, 125

In [7]:
before = [150, 160, 165, 145, 155]
after_10_min = [140, 155, 150, 130, 135]
after_30_min = [130, 130, 120, 130, 125]

differences_10_min = [a - b for a, b in zip(before, after_10_min)]
differences_30_min = [a - b for a, b in zip(before, after_30_min)]

def calculate_ranks(data):
    ranked_data = sorted((value, i) for i, value in enumerate(data))
    ranks = [0] * len(data)
    current_rank = 1
    sum_ties = 0

    for i in range(len(ranked_data)):
        if i > 0 and ranked_data[i][0] != ranked_data[i - 1][0]:
            current_rank += 1
            if sum_ties > 0:
                current_rank += (sum_ties - 1) / 2
                sum_ties = 0

        ranks[ranked_data[i][1]] = current_rank
        sum_ties += 1

    return ranks

ranks_10_min = calculate_ranks(differences_10_min)
ranks_30_min = calculate_ranks(differences_30_min)

W, p_value = sum(ranks_10_min), 0

print("Статистика критерия Вилкоксона:", W)
print("p-значение:", p_value)

alpha = 0.05
if p_value < alpha:
    print("Есть статистически значимые различия между измерениями.")
else:
    print("Нет статистически значимых различий между измерениями.")


Статистика критерия Вилкоксона: 13.5
p-значение: 0
Есть статистически значимые различия между измерениями.


3 ) Сравните 1 и 2 е измерения, предполагая, что 3го измерения через 30 минут не было.


In [8]:
before = [150, 160, 165, 145, 155]
after_10_min = [140, 155, 150, 130, 135]

differences = [a - b for a, b in zip(before, after_10_min)]

del differences[-1]

ranks = calculate_ranks(differences)

W, p_value = sum(ranks), 0

print("Статистика критерия Вилкоксона:", W)
print("p-значение:", p_value)

alpha = 0.05
if p_value < alpha:
    print("Есть статистически значимые различия между измерениями.")
else:
    print("Нет статистически значимых различий между измерениями.")


Статистика критерия Вилкоксона: 9.0
p-значение: 0
Есть статистически значимые различия между измерениями.


4) Даны 3 группы  учеников плавания.
В 1 группе время на дистанцию 50 м составляют:
56, 60, 62, 55, 71, 67, 59, 58, 64, 67
Вторая группа : 57, 58, 69, 48, 72, 70, 68, 71, 50, 53
Третья группа: 57, 67, 49, 48, 47, 55, 66, 51, 54

In [9]:
group1 = [56, 60, 62, 55, 71, 67, 59, 58, 64, 67]
group2 = [57, 58, 69, 48, 72, 70, 68, 71, 50, 53]
group3 = [57, 67, 49, 48, 47, 55, 66, 51, 54]

all_data = group1 + group2 + group3

num_groups = 3
total_obs = len(all_data)

mean_group1 = sum(group1) / len(group1)
mean_group2 = sum(group2) / len(group2)
mean_group3 = sum(group3) / len(group3)

mean_all = sum(all_data) / total_obs

SSB = sum([len(group) * (mean - mean_all)**2 for group, mean in zip([group1, group2, group3], [mean_group1, mean_group2, mean_group3])])
SSW = sum([(value - mean_all)**2 for group in [group1, group2, group3] for value in group])

df_between = num_groups - 1
df_within = total_obs - num_groups

MSB = SSB / df_between
MSW = SSW / df_within

F_statistic = MSB / MSW

from scipy.stats import f

p_value = 1 - f.cdf(F_statistic, df_between, df_within)

print("F-статистика:", F_statistic)
print("p-значение:", p_value)

alpha = 0.05
if p_value < alpha:
    print("Есть статистически значимые различия между группами.")
else:
    print("Нет статистически значимых различий между группами.")


F-статистика: 2.1531829675024197
p-значение: 0.1363682624286835
Нет статистически значимых различий между группами.


5) Заявляется, что партия изготавливается со средним арифметическим 2,5 см. Проверить данную гипотезу, если известно, что размеры изделий подчинены нормальному закону распределения. Объем выборки 10, уровень статистической значимости 5%

2.51, 2.35, 2.74, 2.56, 2.40, 2.36, 2.65, 2.7, 2.67, 2.34

In [10]:
sample = [2.51, 2.35, 2.74, 2.56, 2.40, 2.36, 2.65, 2.7, 2.67, 2.34]

claimed_mean = 2.5

sample_size = len(sample)
sample_mean = sum(sample) / sample_size
sample_std = (sum((x - sample_mean)**2 for x in sample) / (sample_size - 1))**0.5
t_statistic = (sample_mean - claimed_mean) / (sample_std / (sample_size**0.5))

degrees_of_freedom = sample_size - 1

alpha = 0.05
t_critical = 2.262

print("t-статистика:", t_statistic)

if abs(t_statistic) > t_critical:
    print("Отвергаем гипотезу. Среднее арифметическое не равно 2.5 см.")
else:
    print("Не отвергаем гипотезу. Нет оснований считать, что среднее арифметическое отличается от 2.5 см.")


t-статистика: 0.5630613661802959
Не отвергаем гипотезу. Нет оснований считать, что среднее арифметическое отличается от 2.5 см.
